In [ ]:
import os
import sys
import asyncio
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from asgiref.sync import sync_to_async
from IPython.display import display

# If running in a Jupyter environment, allow nested event loops
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

# Django setup
notebook_dir = Path().absolute()
sys.path.insert(0, str(notebook_dir))
if 'DJANGO_SETTINGS_MODULE' in os.environ:
    del os.environ['DJANGO_SETTINGS_MODULE']
os.environ['DJANGO_SETTINGS_MODULE'] = 'inteldocs.settings'
import django
django.setup()

from app.services.vectorstore import vector_store
from langchain_community.retrievers import ContextualCompressionRetriever
from langchain_community.retrievers.document_compressors import LLMListwiseRerank
from langchain_ollama import ChatOllama
from app.models import Document

# Ensure full titles are shown and no wrapping
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', None)

async def get_title(doc_id):
    try:
        d = await sync_to_async(Document.objects.get, thread_sensitive=False)(id=doc_id)
        return d.title
    except Document.DoesNotExist:
        return None

async def process_query(q, top_k, score_threshold, llm_compressor, base_store):
    # 1) Original—pull top_k with scores
    orig_pairs = base_store.similarity_search_with_score(q, k=top_k)
    # 2) Threshold—filter those by score
    thresh_pairs = [(doc, score) for doc, score in orig_pairs if score >= score_threshold]
    # 3) Compressed—just docs, no scores
    compressed_docs = llm_compressor.invoke(q)

    # Map doc_id and chunk_index to score for retrieving compressed scores
    score_map = {
        (doc.metadata.get("doc_id"), doc.metadata.get("index")): score
        for doc, score in orig_pairs
    }

    # Counts for bar chart
    counts = {
        f"Original (k={top_k})": len(orig_pairs),
        f"Threshold (≥{score_threshold})": len(thresh_pairs),
        "Compressed (LLM)": len(compressed_docs),
    }

    # Build flat records list
    records = []
    for name, entries in [
        (f"Original (k={top_k})", orig_pairs),
        (f"Threshold (≥{score_threshold})", thresh_pairs),
    ]:
        for doc, score in entries:
            title = await get_title(doc.metadata.get("doc_id"))
            records.append({
                "Retriever": name,
                "Title": title,
                "Chunk Index": doc.metadata.get("index"),
                "Score": score,
            })
    for doc in compressed_docs:
        doc_id = doc.metadata.get("doc_id")
        chunk_index = doc.metadata.get("index")
        title = await get_title(doc_id)
        score = score_map.get((doc_id, chunk_index))
        records.append({
            "Retriever": "Compressed (LLM)",
            "Title": title,
            "Chunk Index": chunk_index,
            "Score": score if score is not None else None,
        })

    # Build DataFrame and MultiIndex
    df = pd.DataFrame(records)
    df = df.sort_values(['Retriever','Score'], ascending=[True, False])
    df_multi = df.set_index(['Retriever','Title','Chunk Index']).sort_index()

    # Print each section in custom order, sorting by score
    print(f"\nResults for query: {q}\n")
    order = [
        "Compressed (LLM)",
        f"Threshold (≥{score_threshold})",
        f"Original (k={top_k})"
    ]
    for name in order:
        print(f"{name}:")
        try:
            section = df_multi.loc[name]
            section_df = section.reset_index()
            # Sort by Score descending, None last
            section_df['Score'] = pd.to_numeric(section_df['Score'], errors='coerce')
            section_df = section_df.sort_values('Score', ascending=False, na_position='last')
            # Render at 100% width in Jupyter
            styled = section_df.style.set_table_attributes('style="width:100%;"')
            display(styled)
        except KeyError:
            print("  (no results)\n")

    # Bar chart
    plt.figure(figsize=(6, 3))
    plt.bar(counts.keys(), counts.values())
    plt.ylabel("Num Docs")
    plt.title(f"Docs Retrieved for "{q}"")
    plt.xticks(rotation=25, ha="right")
    plt.tight_layout()
    plt.show()

async def main():
    TOP_K = 10
    SCORE_THRESHOLD = 0.4

    llm = ChatOllama(model="llama3.1:8b", base_url="http://ollama:11434", temperature=0)
    compressor = LLMListwiseRerank.from_llm(llm)
    score_retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": SCORE_THRESHOLD}
    )
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=score_retriever
    )

    queries = [
        "Who is John Lenard Requina?"\
    ]
    for q in queries:
        await process_query(q, TOP_K, SCORE_THRESHOLD, compression_retriever, vector_store)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())